In [1]:
import pandas as pd
import numpy as np
import re
from nltk.stem.snowball import RussianStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [2]:
data = pd.read_csv('news.csv')

In [4]:
data = data.dropna(subset=['rubric'])

In [3]:
for c in data['rubric'].unique():
    print(c, sum(data['rubric'] == c))

Экономика 420
Спорт 494
Из жизни 197
Интернет и СМИ 211
Культура 206
Дом 123
Бывший СССР 394
69-я параллель 27
Мир 718
Наука и техника 208
Путешествия 183
Россия 908
Нацпроекты 53
Силовые структуры 220
Ценности 141
nan 0


In [5]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder()
y = ohe.fit_transform(np.array(data['rubric']).reshape(-1, 1)).astype('int').toarray()
column_to_category = ohe.categories_

In [6]:
text = np.array(data['text'])

stem_rus = RussianStemmer(False)

for i in range(len(text)):
    out = ''
    for word in re.sub('[^{0-9а-яА-Я}]', ' ', text[i]).lower().split(' '):
        out += stem_rus.stem(word) + ' '
    text[i] = out

In [7]:
basic_vectorizer = CountVectorizer(min_df=4)
text_train = text[:4000]
text_test = text[4000:]
X_train = basic_vectorizer.fit_transform(text_train)
X_test = basic_vectorizer.transform(text_test)
y_train = y[:4000, -1]
y_test = y[4000:, -1]
# В исходном датасете статьи были перемешаны, поэтому можно разделить даже так

In [11]:
from ensembles import GradientBoostingMSE
from sklearn.metrics import mean_squared_error

grid = {
    'n_estimators': [3, 5, 7, 10, 15],
    'max_depth': [1, 2, 3, 4, 5, 6]
}

gs_gbmse = GridSearchCV(estimator=GradientBoostingMSE(), param_grid=grid, scoring='neg_mean_squared_error')
# GradientBoostingMSE - не библиотечный класс, но он удовлетворяет scikit API
gs_gbmse.fit(X_train, y_train)
gs_gbmse.best_params_

{'max_depth': 2, 'n_estimators': 5}

In [17]:
best_max_depth, best_n_estimators = gs_gbmse.best_params_.values()

In [12]:
np.mean(y_train)

0.09375

In [14]:
mean_squared_error(gs_gbmse.predict(X_test), y_test), mean_squared_error(np.ones_like(y_test) * np.mean(y_train), y_test)

(0.0496689604100579, 0.08147792929920478)

Делаем вывод, что с определением статей на экономическую тематику (последний столбец отвечал за тему "Экономика") алгоритм обучения справляется (неидеально, но всё же лучше, чем константный прогноз). Можем сохранять.